# Guided Lab — Chapter 3: Inheritance, Composition, and Polymorphism
  
**Based on:** Chapter 3, *Python Object-Oriented Programming* (Lott & Phillips, 4th ed.)   
**Estimated Time:** 90–120 minutes  

---

## Lab Purpose

This guided lab is designed to help you practice *design judgment* in Python OOP by progressively building a small system that demonstrates:

- When inheritance is appropriate  
- When composition is preferred  
- How abstract base classes define contracts  
- How mixins add orthogonal behavior  
- How method resolution order (MRO) affects runtime behavior  
- How polymorphic dispatch eliminates type-based logic

---

## Scenario Overview: A Notification System

You are building a notification subsystem for an application. The system must:

- Support multiple delivery channels (Email, SMS, Slack)
- Allow new channels to be added without rewriting existing code
- Avoid type-checking conditionals
- Be easy to test and reason about

We will evolve the design step by step, intentionally surfacing the design decisions discussed in Chapter 3.

---

## Part 1 — Defining a Stable Contract with an Abstract Base Class

### Step 1.1 — Create the Sender Contract

We begin by defining the *behavioral contract* that all notification channels must satisfy.

**Instructions:**
1. Import the required ABC utilities.
2. Define an abstract base class called `Sender`.
3. Add a single abstract method called `send()`.

Run the cell and verify you **cannot instantiate `Sender`** directly.

In [ ]:
from abc import ABC, abstractmethod

class Sender(ABC):
    @abstractmethod
    def send(self, recipient: str, message: str) -> None:
        """Send a message to a recipient."""
        raise NotImplementedError

### Checkpoint
- You should **not** be able to instantiate `Sender` directly.
- The contract should be minimal and focused.

Try this and confirm it raises a `TypeError`:
```python
Sender()
```

In [ ]:
Sender()

---

## Part 2 — Implementing Concrete Senders (Polymorphism Begins)

### Step 2.1 — Implement `EmailSender`

**Instructions:**
- Create a class `EmailSender` that inherits from `Sender`.
- Implement the `send()` method.

In [ ]:
class EmailSender(Sender):
    def send(self, recipient: str, message: str) -> None:
        print(f"EMAIL → {recipient}: {message}")

### Step 2.2 — Implement `SmsSender`

In [ ]:
class SmsSender(Sender):
    def send(self, recipient: str, message: str) -> None:
        print(f"SMS → {recipient}: {message}")

### Checkpoint
- Both classes should be instantiable.
- Both should satisfy the same interface.
- Neither should require changes to the `Sender` base class.

Run the test cell below.

In [ ]:
EmailSender().send("alice@example.com", "Hello via email")
SmsSender().send("555-1234", "Hello via SMS")

---

## Part 3 — Using Composition for Orchestration

### Step 3.1 — Create the `Notifier`

Rather than creating specialized notifier subclasses, we will *compose* behavior.

**Instructions:**
- Create a `Notifier` class.
- Inject one or more `Sender` objects.
- Delegate notification responsibility to each sender.

In [ ]:
from typing import Iterable

class Notifier:
    def __init__(self, senders: Iterable[Sender]):
        self._senders = list(senders)

    def notify(self, recipient: str, message: str) -> None:
        for sender in self._senders:
            sender.send(recipient, message)

### Step 3.2 — Test Composition

Run the following. You should see **two outputs** (email + SMS).

In [ ]:
notifier = Notifier([EmailSender(), SmsSender()])
notifier.notify("alice@example.com", "Your report is ready")

### Reflection (write brief answers)
1. Why is composition preferable here to inheritance?  
2. What would an inheritance-based design look like?  
3. What changes if we add a third sender?

Write your answers in the markdown cell below.

1. It iss just more flexible. Instead of making new classes for every combination, we can just put different senders in a list whenever we want. It keeps the code clean and we don't end up with way too many classes.

2. What inheritance would look like: We would have to create a specific class for everything. Like, one class for EmailOnly, one for SmsOnly, and another one for EmailAndSms. It will be a mess very quickly and will be hard to manage.

3. Adding a third sender: It is easy. We just create the new sender (like Slack) and give it to the Notifier. We dont have to touch or change any of the code we already wrote for the other senders.

---

## Part 4 — Dependency Injection as a Design Pattern

### Step 4.1 — Understand the Boundary

Notice that `Notifier`:
- Does not know *which* senders it uses
- Does not construct its own dependencies
- Depends only on the `Sender` abstraction

This is **dependency injection**, enabled by composition.

### Step 4.2 — Swap Implementations

Run the following and confirm that **Notifier does not change**, only the injected dependency changes.

In [ ]:
notifier = Notifier([SmsSender()])
notifier.notify("555-1234", "System maintenance tonight")

### Reflection
1. What changed in the system?  
2. What did *not* change?  
3. Why does that matter for maintainability?

Write your answers below.


1. Only the SmsSender changed instead of EmailSender.
2. The Notifier class is still unchanged.
3. This matters for maintainability because we can add/remove/swap implementations without touching the main old logic like orchestration code, it also couses reducing bugs and making testing easier.

---

## Part 5 — Adding Behavior with Mixins

### Step 5.1 — Create a `LoggingMixin`

We want to add logging *without modifying* existing sender classes.

Key idea: a safe mixin is focused, cooperative, and uses `super()` to chain behavior.

In [ ]:
class LoggingMixin:
    def send(self, recipient: str, message: str) -> None:
        print(f"LOG: sending to {recipient}")
        return super().send(recipient, message)

### Step 5.2 — Combine the Mixin with a Sender

We create a new class by combining the mixin with an existing sender.

Important: The mixin must appear **before** the concrete sender class so its `send()` runs first.

In [ ]:
class LoggedEmailSender(LoggingMixin, EmailSender):
    pass

### Step 5.3 — Test the Behavior

In [ ]:
sender = LoggedEmailSender()
sender.send("alice@example.com", "Build completed")

### Reflection
1. Why must `LoggingMixin` appear before `EmailSender`?  
2. Why does it call `super()` instead of `EmailSender.send(...)` directly?  
3. What could go wrong if the mixin assumed state (attributes) that don’t exist?

Write your answers below.


1. also  because of how Python looks for methods (MRO) process left to rigth, If we put the Mixin first, Python finds its send() method first. This way, it can print the log message before the actual email gets sent..
2. Using "suoper" is better because it makes the Mixin flexible. It doesn't lock the Mixin to just one class. It allows it to work with any sender like sms or slack without changing the Mixin's code.
3, The program would probably crash with an error. If the Mixin tries to use a variable that the main class doesnt have, Python wont know what to do and will stop the code.

---

## Part 6 — Exploring Method Resolution Order (MRO)

### Step 6.1 — Inspect the MRO

Run the following and interpret the output.

In [ ]:
print(LoggedEmailSender.__mro__)

### Discussion Prompts
- In what order are methods resolved?
- Where does `object` appear?
- How does this enable cooperative multiple inheritance?

Write a short explanation below.

 
Methods called from left to right. Python first checks LoggingMixin class, then EmailSender class and finally the Sender class.
The object class is always at the very end because it is the base of everything in Python.
This helps "cooperative inheritance" because each class uses "super" to pass the work to the next class in line. It creates a chain where every class can do its job without breaking the system.

---

## Part 7 — Adding Another Mixin (Retry Behavior)

### Step 7.1 — Implement `RetryMixin`

This mixin wraps `send()` with a simple retry loop. In real systems, you would target specific exceptions and use exponential backoff.

In [ ]:
class RetryMixin:
    def send(self, recipient: str, message: str) -> None:
        for attempt in range(1, 4):
            try:
                return super().send(recipient, message)
            except Exception as e:
                print(f"Retry {attempt} failed: {e}")
        raise RuntimeError("All retries failed")

### Step 7.2 — Combine Multiple Mixins

Order matters. Here, `RetryMixin` wraps `LoggingMixin`, which wraps `EmailSender`.

In [ ]:
class ReliableEmailSender(RetryMixin, LoggingMixin, EmailSender):
    pass

### Step 7.3 — Inspect the New MRO

In [ ]:
print(ReliableEmailSender.__mro__)

### Reflection
1. Which `send()` executes first?  
2. Describe the chain of calls caused by `super()`.  
3. What does “cooperative multiple inheritance” mean in this context?

Write your answers below.


1. The sender method in RetryMixin runs first because it is the first class listed in the parentheses when we defined ReliableEmailSender.

2. As it works like a chain so  RetryMixin calls "super" which goes to LoggingMixin, then LoggingMixin calls super which finally goes to the real EmailSender.
3. It means all the classes "cooperate" by using "super" . instead of one class taking over and stopping the process, they each do their small job and then pass the work to the next class.

---

## Part 8 — Polymorphic Dispatch in Action

### Step 8.1 — Add a New Channel (`SlackSender`)

The key test: we should be able to add this without editing `Notifier`.

In [ ]:
class SlackSender(Sender):
    def send(self, recipient: str, message: str) -> None:
        print(f"SLACK → @{recipient}: {message}")

### Step 8.2 — Use All Senders Together

Run the following. You should see three outputs (email + SMS + Slack).

In [ ]:
notifier = Notifier([
    ReliableEmailSender(),
    SmsSender(),
    SlackSender()
])

notifier.notify("alice", "Deployment successful")

---

## Final Reflection (Design Judgment)

Answer the following in 5–8 sentences each:

1. Where does polymorphic dispatch occur in this lab?  
2. Which parts of the system remained unchanged as we added new channels and behaviors?  
3. Map each concept to a concrete artifact:
   - Inheritance  
   - Composition  
   - ABCs  
   - Mixins  
   - MRO  
   - Polymorphic dispatch  
4. If you had to productionize this system, what would you improve first (and why)?

1. it happens in the Notifier class inside the "notify" method. When we run the loop and call" sender.send()".Python automatically knows which version to use (Email or SMS) without us needing any if statements or conditional statements

2.  Sender class and the Notifier class never changed. Even when we added new things like Slack or Mixins for logging, we didnot have to touch the main code that handles the notifications.
3. 
- Inheritance: When EmailSender and SmsSender get their structure from Sender.

- Composition: When Notifier keeps a list of different sender objects to use them together.

- ABCs: The Sender class, which acts as a "rulebook" for all other senders.

- Mixins: LoggingMixin and RetryMixin that add extra features like a new plugin.

- MRO: The order Python follows (left to right) to find the right "send" method.

- Polymorphic dispatch: The part where one line of code (sender.send) works for many different types of objects.

4. The first thing I improve is Error Handling. for now if one sender fails, the whole program stop. Iill make the system Async. Sending notifications can be slow, and using asyncio allows the program to send many messages at the same time without waiting for each one to finish.

---

## Lab Wrap-Up

By completing this lab, you have:

- Used inheritance intentionally (ABCs, mixins)
- Preferred composition for orchestration
- Leveraged polymorphism to eliminate conditionals
- Understood how MRO governs runtime behavior

This lab is the practical counterpart to Chapter 3’s design philosophy.